Titanic survival prediction

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

input/train.csv
input/gender_submission.csv
input/test.csv


In [3]:


titanic_data = pd.read_csv("input/train.csv")
#titanic_data.head()

print("length titanic data: "+str(len(titanic_data)))

def shuffle_and_split_data(data, test_ratio):
    # np.random.seed(42) pour obtenir des permutations prévisibles
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

train_set, test_set = shuffle_and_split_data(titanic_data, 0.2)

print(f"length titanic data: {str(len(train_set))}")
train_set.head()

length titanic data: 891
length titanic data: 713


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
331,332,0,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S
733,734,0,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S
382,383,0,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S
704,705,0,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S


In [4]:
women = train_set.loc[train_set.Sex == 'female']["Survived"]
print (women)
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

10     1
813    0
865    1
259    1
486    1
      ..
330    1
700    1
71     0
106    1
435    1
Name: Survived, Length: 246, dtype: int64
% of women who survived: 0.7398373983739838


Les femmes semblent avoir de meilleures chances de survie. Mais explorons d'autres facteurs

In [7]:
corr_matrix=train_set.corr(numeric_only=True)
corr_matrix["Survived"].sort_values(ascending=False)

Survived       1.000000
Fare           0.245787
Parch          0.079601
PassengerId    0.016849
SibSp         -0.046857
Age           -0.063162
Pclass        -0.319763
Name: Survived, dtype: float64

Corrélation positive entre le tarif payé et les chances de survie
Les passagers de classe plus élevée ont aussi plus de chance de survie

In [9]:
women_1st = train_set.loc[train_set.Sex == 'female'][train_set.Pclass==1]["Survived"]
print (women_1st)
rate_women = sum(women_1st)/len(women_1st)

print("% of women of 1st class who survived:", rate_women)

486    1
716    1
781    1
412    1
742    1
      ..
166    1
504    1
871    1
700    1
435    1
Name: Survived, Length: 70, dtype: int64
% of women of 1st class who survived: 0.9571428571428572


/tmp/ipykernel_3599/1945792378.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  women_1st = train_set.loc[train_set.Sex == 'female'][train_set.Pclass==1]["Survived"]


95% des femmes de la 1e classe ont survécu

In [11]:
# axis=1 -> traite ligne par ligne. Si axis = 0, colonne par colonne
# any , renvoie true pour n'importe quelle valeur de la ligne
null_rows_id = train_set.isnull().any(axis=1)